<b>Connecting CatSim to SysArch</b>

In order to ensure that you can connect the CatSim code to the SysArch database, you must edit/create the file `db-auth.paf` in the directory `$HOME/.lsst` on your machine.  The format and contents of the `db-auth.paf` file can be found under Step (3) of this confluence page:

https://confluence.lsstcorp.org/display/SIM/Accessing+the+UW+CATSIM+Database

<b> Setting up CatSim </b>

Before running this notebook, you will need to install the LSST Simulations stack.  Follow the instructions here


https://confluence.lsstcorp.org/display/SIM/Catalogs+and+MAF


<b>Exploring SysArch</b>

The LSST package `sims_catalogs_generation` provides the class `DBObject` which can connect to an arbitrary database, inspect its tables and columns, and run queries on the tables.  We will use that class to explore the SysArch databse.

In [1]:
from lsst.sims.catalogs.generation.db import DBObject
dbo = DBObject(database='sysarch', host='terminal.lsst.org', port='3306',
               driver='mysql')

Now that we have a connection to the SysArch database, we can inspect its tables.  The following cell yields a list of all of the table names in SysArch.

In [2]:
tableNameList = dbo.get_table_names()

In [3]:
for tableName in tableNameList:
    print tableName

t_attribute
t_attributeconstraints
t_attributetag
t_authors
t_cardinality
t_category
t_clients
t_complexitytypes
t_connector
t_connectorconstraint
t_connectortag
t_connectortypes
t_constants
t_constrainttypes
t_datatypes
t_diagram
t_diagramlinks
t_diagramobjects
t_diagramtypes
t_document
t_ecf
t_efforttypes
t_files
t_genopt
t_glossary
t_html
t_image
t_implement
t_issues
t_lists
t_mainttypes
t_method
t_metrictypes
t_object
t_objectconstraint
t_objecteffort
t_objectfiles
t_objectmetrics
t_objectproblems
t_objectproperties
t_objectrequires
t_objectresource
t_objectrisks
t_objectscenarios
t_objecttests
t_objecttrx
t_objecttypes
t_ocf
t_operation
t_operationparams
t_operationposts
t_operationpres
t_operationtag
t_package
t_palette
t_paletteitem
t_phase
t_primitives
t_problemtypes
t_projectroles
t_propertytypes
t_requiretypes
t_resources
t_risktypes
t_roleconstraint
t_rtf
t_rtfreport
t_rules
t_scenariotypes
t_script
t_secgroup
t_secgrouppermission
t_seclocks
t_secpermission
t_secpolicies
t_s

We can also learn about the columns in each table.  The following produces a dict of lists.  The dict is keyed to the names of tables on SysArch.  The contents of the dict list the columns in each table.

In [4]:
columnNameDict = dbo.get_column_names()

In [5]:
for col in columnNameDict['t_object']:
    print col

Object_ID
Object_Type
Diagram_ID
Name
Alias
Author
Version
Note
Package_ID
Stereotype
NType
Complexity
Effort
Style
Backcolor
BorderStyle
BorderWidth
Fontcolor
Bordercolor
CreatedDate
ModifiedDate
Status
Abstract
Tagged
PDATA1
PDATA2
PDATA3
PDATA4
PDATA5
Concurrency
Visibility
Persistence
Cardinality
GenType
GenFile
Header1
Header2
Phase
Scope
GenOption
GenLinks
Classifier
ea_guid
ParentID
RunState
Classifier_guid
TPos
IsRoot
IsLeaf
IsSpec
IsActive
StateFlags
PackageFlags
Multiplicity
StyleEx
ActionFlags
EventFlags


There are obviously a large number of tables in SysArch.  We only care about four of them.


* `t_object` -- Everything that exists as an object in SysML (every constraint, every package, every requirement, etc.) is listed in this table.  Each object has a unique `Object_ID` in addition to its name (which does not have to be unique)


* `t_objectproperties` -- This table contains the properties, if any, associated with the objects in `t_object`.  The rows in `t_object` contain the same `Object_ID` as `t_object` so, once you have an object's ID from `t_object`, you can look up its properties in `t_objectproperties`.


* `t_attribute` -- This table contains the attributes associated with the objects in `t_object`.  The attributes are the quantitative values associated with an object.  Again, attributes in `t_attribute` are associated with an `Object_ID`, so we can look them up once we have looked up the object in `t_object`.


* `t_connector` -- This table lists all of the connections (e.g. 'refines', 'trace', etc.) between objects.  Each connection contains a column `Start_Object_ID` and `End_Object_ID` indicating which two objects it connects.  Furthermore, objects that 'refine' another object have a column `ParentID` in `t_object` referring to the `Object_ID` of the object the refine.  This allows us to find all of the relationships between objects.

To summarize: any useful query must begin by looking for a `Name` in `t_object` and associating that with an `Object_ID` and possibly a `ParentID`.  Then, quantitative attributes must be found by searching for that same `Object_ID` in `t_objectproperties` and `t_attribute`.  Finally, relationships between objects must be found by searching the `Start_Object_ID` and `End_Object_ID` columns of `t_connector`.

In [6]:
import numpy as np

In [7]:
dtype = np.dtype([('Name', str, 300), ('Object_ID', np.int),
                  ('ParentID', np.int), ('Author', str, 100),
                 ('Version', str, 100)])

Let's query for all objects whose names contain 'm1' in them

In [8]:
query = "select t.Name, t.Object_ID, t.ParentID, " \
        +"t.Author, t.Version from t_object t where t.Name like '%m1%'"

In [9]:
results = dbo.execute_arbitrary(query, dtype=dtype)
print results

[('M1M3 Surface Deformation', 251817, 0, 'Chuck Claver', '1.0')
 ('pM1M3', 313098, 313093, 'fdelgado', '1.0')
 ('M1M3', 313149, 313189, 'jsebag', '1.0')
 ('M1M3Dummy', 313158, 313149, 'jsebag', '1.0')
 ('M1M3Controller', 313202, 313149, 'Francisco Delgado', '1.0')
 ('M1M3Support', 313227, 313149, 'jsebag', '1.0')
 ('M1M3Thermal', 313244, 313149, 'jsebag', '1.0')
 ('M1M3Container', 313259, 313149, 'jsebag', '1.0')
 ('M1M3Cell', 313265, 313149, 'jsebag', '1.0')
 ('M1M3Mirror', 313281, 313149, 'jsebag', '1.0')
 ('M1 Prescription', 315388, 0, 'Brian Selvy', '1.0')
 ('Check Temperature in M1M3 Cell', 316765, 316726, 'Jacques Sebag', '1.0')
 ('CO2 Clean M1M3', 316795, 316742, 'Chuck Claver', '1.0')
 ('$uicmd=MenuCmd;param1=GenerateMDGTechnology;', 328283, 0, 'Brian Selvy', '1.0')
 ('$uicmd=ShowView;param1=ViewDiagramToolBox', 328294, 0, 'Brian Selvy', '1.0')
 ('Control M1M3 Temperature', 336923, 0, 'Paul Lotz', '1.0')
 ('Control M1M3 Shape', 336933, 0, 'Paul Lotz', '1.0')
 ('Control M1M3 Pos

<b>Problems with the current EA configuration</b>

<b>Degenerate entries:</b> There are multiple objects in `t_boject`  that share the same name.  Sometimes they have different `author` or `Version` numbers listed, so the degeneracy can be broken.  Sometimes they do not, for example:

In [10]:
query = "select t.Name, t.Object_ID, t.ParentID, " \
        +"t.Author, t.Version from t_object t where t.Name = 'm1Prescription'"
results = dbo.execute_arbitrary(query, dtype=dtype)
print results

[('m1Prescription', 385259, 385322, 'Chuck Claver', '1.0')
 ('m1Prescription', 385875, 385937, 'Chuck Claver', '1.0')]


<b>Version Control:</b> While `Version` is a column in `t_object`, it does not appear to correspond to anything.  Only a few entries in `t_object` have a `Version` other than `1.0`.

In [11]:
dtype = np.dtype([('Name', str, 100), ('Version', str, 100)])
query = "select t.Name, t.Version from t_object t where t.Version!='1.0'"
results = dbo.execute_arbitrary(query, dtype=dtype)
print results

[('Science Usage and Analysis', '1.1') ('Actors', '1.1')
 ('Raw Exposure Processing', '1.1') ('Association', '1.1')
 ('DIA Source Detection and Characterization', '1.1')
 ('Alert Subscription', '1.1') ('Database Schema', '3.3.2')
 ('02C.05 Science  User Interface and Analysis Tools', '1.1')
 ('02C.03.02 Association Pipelines', '1.1')
 ('02C.03.01 Image Processing Pipeline', '1.1')
 ('02C.03.07 Difference Imaging Pipelines', '1.1')
 ('02C.04.05 Deep Detection Pipeline', '1.1')
 ('DMS Component and Deployment Model', '1.1')
 ('LSE-131: Interface between Data Management and EPO', '2.0')
 ('Level 1 Data Quality Report Definition', '1.1')]


Reading on-line documentation leads me to believe (and Mario Juric agrees) that version control in Enterprise Architect is handled with actual version control software like SVN or CVS.  This means that the databases can only contain one version of the project at a time.  It is not yet clear to me that we can write our own interface to the SVN or CVS repositories underlying our EA setup.

<b>Custom Interface</b>

I have written a small module `EADBWrapper` (contained in this repository) that contains classes specifically for interfacing with SysArch.  Classes included in this module are


* `EADBWrapper` -- This class wraps `DBObject` with methods specifically designed to query the tables on SysArch


* `SysMLObject` -- This class uses `EADBWrapper` to query SysArch and store all of the relevant information about an object (it's properties and attributes)


* `SysMLObjectList` -- This class stores a list of `SysMLObject`s and scrapes the database for information about their relationships

`EADBWrapper` contains a method `writeFamilyTree` which takes an object name, finds all of the object's related to it, and prints out their attributes and relationships.  We demonstrate below.

In [12]:
from EADBWrapper import EADBWrapper

In [13]:
eadbo = EADBWrapper()

<b>Note: The cell below will fail</b> because more than one object exists with the specified name.  We can, in this case, break the degeneracy by specifying an author.

In [14]:
eadbo.writeFamilyTree('g-band Response Envelope')

RuntimeError: More than one object match the name you gave. Try specifying an author or a version

In [15]:
eadbo.writeFamilyTree('g-band Response Envelope', author='Chuck Claver')

Name: g-band Response Envelope
Version: 1.0
Author: Chuck Claver
Property: LSSTRequirements
Value: OSS-REQ-0241

Name: g-InBandLimits
Version: 1.0
Author: Chuck Claver
Property: isEncapsulated
Value: None
Values: true,false

Relationships:
     �refine� g-band Response Envelope

Attributes:
    ------------
    g_InBandRed = 537.0 nm
    The in-band red limit for the g-band filter response normalization.
    ------------
    g_InBAndBlue = 416.5 nm
    The in-band blue limit for the g-band filter response normalization.

Name: Optical Sensitivity and Performance : Filter Response
Version: 1.0
Author: Chuck Claver

Name: g_lowerEnvelope
Version: 1.0
Author: Chuck Claver
Property: isEncapsulated
Value: None
Values: true,false

Relationships:
     �refine� g-band Response Envelope

Attributes:
    ------------
    g_lowerBlue(0) = 391.5 nm
    <font color="#0f0f0f">The blue side zero response wavelength of the g-band lower envelope.</font>
    ------------
    g_lowerRed(0.97) = 537.75 nm